In [2]:
import os
import numpy as np

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Using TensorFlow backend.


# Generate dataset 

(->generate_data_samples.py)

## Import dataset

In [3]:

batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()


## Load into numpy array files

In [4]:
root_path = os.path.expanduser("~/python-projects/substra/substra/examples/mnist")
data_path = os.path.join(root_path, 'data')
train_data_path = os.path.join(data_path, 'train')
test_data_path = os.path.join(data_path, 'test')
assets_path = os.path.join(root_path, 'assets')

print(data_path)

OUT_FILE = {
    os.path.join("train_data","x_train.npy"): x_train,
    os.path.join("train_data","y_train.npy"): y_train,
    os.path.join("test_data", "x_test.npy"): x_test,
    os.path.join("test_data", "y_test.npy"): y_test,

}

for filename, data in OUT_FILE.items():
    full_path = os.path.join(assets_path,filename)
    os.makedirs(os.path.dirname(full_path), exist_ok=True)
    np.save(full_path, data)

/home/fabien/python-projects/substra/substra/examples/mnist/data


# Algorithm

-> algo.py

## Normalize data


In [5]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


## Create model

In [6]:

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])



## Train model

TODO : opener

In [7]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
 

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
21504/60000 [=========>....................] - ETA: 4:09 - loss: 0.4701 - accuracy: 0.8479

KeyboardInterrupt: 

## Test model

In [9]:
         
score = model.evaluate(x_test, y_test, verbose=0) #split fct en 2 parts pour faire la metrics
print('Test loss:', score[0])
print('Test accuracy:', score[1])